In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unrar x '/content/drive/MyDrive/CV5-Homework.rar'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/CV5-Homework.rar

Creating    utils                                                     OK
Extracting  utils/coco_eval.py                                             0%  OK 
Extracting  utils/coco_utils.py                                            0%  OK 
Extracting  utils/engine.py                                                0%  OK 
Creating    utils/fix1_model                                          OK
Extracting  utils/fix1_model/fix1_ports_model.pth                          0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46%

In [4]:
%cd utils

/content/utils


In [5]:
!chdir utils

/bin/bash: chdir: command not found


In [6]:
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd
import os

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

from PIL.ExifTags import TAGS
from engine import train_one_epoch, evaluate
import utils
from torchvision import transforms as T 

In [7]:
class DataSet(torch.utils.data.Dataset):
    
    def __init__(self,root, data_file, transform):
        self.root = root ### mapa cu fotografii
        self.transform = transform
        self.data = pd.read_csv(data_file, sep=';')
        self.imgs = self.data['image_path'].values
        self.path_to_data_file = data_file
        
    def __getitem__(self,idx):
        
        img_path = os.path.join(self.root,self.imgs[idx])
        img = Image.open(img_path).convert('RGB')
        
        
        for orientation in TAGS.keys() : 
            if TAGS[orientation]=='Orientation' : break 
        # Getting the exif
        exif=dict(img.getexif().items())
        # Rotating the image if the orientation is wrong.
        if len(exif)!=0:                                                             
            if orientation in exif.keys():
                if exif[orientation] == 3 : 
                    img=img.rotate(180, expand=True)
                elif exif[orientation] == 6 : 
                    img=img.rotate(270, expand=True)
                elif exif[orientation] == 8 : 
                    img=img.rotate(90, expand=True)
                    
        box_list = self.data[self.data['image_path']==self.imgs[idx]][['x_min','y_min','x_max','y_max']].values
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_obj = len(box_list)
        labels = torch.ones((num_obj,), dtype = torch.int64)
        iscrowd = torch.zeros((num_obj,), dtype=torch.int64)
        area = (boxes[:,3] - boxes[:, 1]) * (boxes[:,2] - boxes[:, 0])
        
        image_id = torch.tensor([idx])
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['area'] = area
        target['iscrowd'] = iscrowd
        target['image_id'] = image_id
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, target
    
    def __len__(self):
        return len(self.data)

In [8]:
transformations = T.Compose([T.ToTensor()])

dataset = DataSet(r'/content/data',
                  r'/content/handguns.csv',
                  transformations)

torch.manual_seed(1)

indices = torch.randperm(len(dataset)).tolist()
train = torch.utils.data.Subset(dataset, indices[:100])
test = torch.utils.data.Subset(dataset, indices[100:])

train_loader = torch.utils.data.DataLoader(train, batch_size = 1, shuffle=True, collate_fn = utils.collate_fn)
test_loader = torch.utils.data.DataLoader(test, batch_size = 1, shuffle=False, collate_fn = utils.collate_fn)

In [9]:
def get_model(num_clase):
    
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    for param in model.parameters():
        param.required_grad = False
        
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_clase)
    
    return model

In [10]:
DEVICE = torch.device('cuda' if torch.cuda.is_available else 'cpu')

num_clase = 2
model = get_model(num_clase).to(DEVICE)

params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(params, lr = 0.005, momentum=0.9, weight_decay = 0.0005)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [13]:
num_epoch = 10

for epoch in range(num_epoch):
    
    train_one_epoch(model, optimizer, train_loader, DEVICE, epoch, print_freq = 10)
    
    evaluate(model, test_loader, device=DEVICE)

Epoch: [0]  [  0/100]  eta: 0:01:06  lr: 0.000055  loss: 0.0972 (0.0972)  loss_classifier: 0.0321 (0.0321)  loss_box_reg: 0.0637 (0.0637)  loss_objectness: 0.0001 (0.0001)  loss_rpn_box_reg: 0.0014 (0.0014)  time: 0.6696  data: 0.0109  max mem: 1196
Epoch: [0]  [ 10/100]  eta: 0:00:54  lr: 0.000560  loss: 0.0342 (0.0617)  loss_classifier: 0.0110 (0.0198)  loss_box_reg: 0.0225 (0.0362)  loss_objectness: 0.0005 (0.0010)  loss_rpn_box_reg: 0.0024 (0.0047)  time: 0.6027  data: 0.0079  max mem: 1196
Epoch: [0]  [ 20/100]  eta: 0:00:47  lr: 0.001065  loss: 0.0454 (0.0592)  loss_classifier: 0.0120 (0.0194)  loss_box_reg: 0.0303 (0.0354)  loss_objectness: 0.0005 (0.0013)  loss_rpn_box_reg: 0.0013 (0.0031)  time: 0.5954  data: 0.0073  max mem: 1196
Epoch: [0]  [ 30/100]  eta: 0:00:41  lr: 0.001569  loss: 0.0494 (0.0601)  loss_classifier: 0.0120 (0.0196)  loss_box_reg: 0.0304 (0.0362)  loss_objectness: 0.0004 (0.0012)  loss_rpn_box_reg: 0.0013 (0.0032)  time: 0.5970  data: 0.0073  max mem: 1196


In [14]:
# Use this code to made a predition.
from PIL import ImageDraw
# Getting the image.
for idx in range(2,20):
    img, _ = test[idx]
    # Getting the object coordinates.
    label_boxes = np.array(test[idx][1]['boxes'])

    # Setting the model to eval state.
    model.eval()
    # Making the prediction.
    with torch.no_grad():
        prediction = model([img.to(DEVICE)])

    # Getting an drawing the image.
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    # Drawing the real box around the object.
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
                       (label_boxes[elem][2], label_boxes[elem][3])],
                      outline='green', width=3)
    # Drawing the predicted box around the object.
    for element in range(len(prediction[0]['boxes'])):
        boxes = prediction[0]['boxes'][element].cpu().numpy()                                     
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)

        if score > 0.2:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])],
                         outline='red', width=3)
            draw.text((boxes[0], boxes[1]), text=str(score))
    display(image)

Output hidden; open in https://colab.research.google.com to view.